In [ ]:
import pandas as pd
import pypsa
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import cartopy.crs as ccrs
import ipynb
import os
from os import walk
import plotly.express as px
import json
import plotly.graph_objs as gobj
import plotly.graph_objects as go

In [ ]:
def load_data(mypath1,filenames):
    networks = {}
    planning_horizon = []
    for file in filenames:
        year = file[-7:-3]
        print("Loading year {} ".format(year))
        networks[year] = pypsa.Network(os.path.join(mypath1,file))
        planning_horizon.append(year)
    print("I have done my deed now set me Free, mortal!")
    return(networks, planning_horizon)

In [ ]:
def get_grid_df(n,carrier):
    """Gets all capacity dataframes for the grid
    Parameters
    ----------
    n : Networks
    carrier : string (== "gas pipeline", "AC", "H2 pipeline")
        carrier (e.g. "gas pipeline", "DC") used in lines and links for transporting a specific energy
    
    Returns
    -------
    two dataframes:
    normal = classic grid, including newly build H2 grid
    retro only used for H2 retrofitted piplines.
    """
    normal = n.copy()
    retro = n.copy()
    dc = n.copy()
    if carrier == 'AC':
        normal.lines = n.lines[n.lines.type =="Al/St 240/40 4-bundle 380.0"]
        dc.links = n.links[n.links.carrier == "DC"]
        for bus in normal.buses[n.buses.carrier=="AC"].index:
            dc.buses.loc[f"{bus}", ["x","y"]] = normal.buses.loc[bus, ["x","y"]]
    else: 
        normal.links = n.links[(n.links.carrier == carrier)]
        carrier_retro = carrier + " retrofit"
        retro.links = n.links[(n.links.carrier == carrier_retro)]
        for bus in normal.buses[n.buses.carrier=="AC"].index:
            normal.buses.loc[f"{bus}"+ " "  + carrier[:3].strip(), ["x","y"]] = normal.buses.loc[bus, ["x","y"]]
            retro.buses.loc[f"{bus}"+ " " + carrier[:3].strip(), ["x","y"]] = retro.buses.loc[bus, ["x","y"]]
    return(normal,retro,dc)

In [ ]:
countries = gpd.read_file(r'C:\Users\johannes.misensky\OneDrive - AGGM\Dokumente\pyp\Shapefile\shape_nuts2.shp')


In [ ]:
mypath = r"C:\Users\johannes.misensky\OneDrive - AGGM\Dokumente\ESM_Aus\postnetworks\test_3"
mypath1 = f"{mypath}"
filenames = next(walk(mypath1), (None, None, []))[2]

networks = {}
planning_horizon = []
filenames

networks, planning_horizon = load_data(mypath1,filenames)

In [ ]:
#countries.to_file("path_to_GeoJSON _file", driver = "GeoJSON")
with open("path_to_GeoJSON _file") as geofile:
    j_file = json.load(geofile)
i = 1    
for feature in j_file["features"]:
        feature ['id'] = str(i).zfill(2)
        i += 1

In [ ]:
# fig = px.choropleth_mapbox(df, geojson=j_file, locations=df['ct'],  featureidkey='properties.COUNTRY_ID',
#                            mapbox_style="carto-positron",
#                            zoom=3, #center = {"lat": 37.0902, "lon": -95.7129},
#                            opacity=0.5,
#                            labels={'unemp':'unemployment rate'}
#                           )

# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


fig = go.Figure(data=go.Choropleth(
    geojson=j_file,
    featureidkey= 'properties.COUNTRY_ID',
    locationmode='geojson-id', # honestly no idea what this does
    #locations=df['ct'],
    z=df['v_nom'], # Placeholder Variable for a later use could be colorcode for powerussage?  
    autocolorscale=False,
    showscale=False,
    hoverinfo = 'none',
    # text = df['ct']

))

fig.update_geos(
    visible=True, resolution=50, 
    scope="europe",
    # center=dict(lon=11, lat=40), # this part is to zone to AT
    # projection_rotation=dict(lon=30, lat=30, roll=30),
    # lataxis_range=[10,11], lonaxis_range=[33,41],
    showcountries=True, countrycolor="Black",
    #showsubunits=True, subunitcolor="Blue"
)
fig.show()

In [ ]:
# carrier = 'AC'
# year = '2020'
# normal, retro, dc = get_grid_df(n = networks[year], carrier = carrier)
# carrier= carrier[:3].strip()          #this part is unnecessary here should be deleted as soon as get_grid_df is modified 
# df = normal.buses[normal.buses.carrier == carrier].copy()
# df['ct'] = df.index.str[:5]
# divisor = 1e3

# fig = go.Figure(data=go.Choropleth(
#     #geojson=j_file['features'],
#     featureidkey= 'properties.COUNTRY_ID',
#     locationmode='geojson-id', # honestly no idea what this does
#     locations=df['ct'],
#     z=df['v_nom'], # Placeholder Variable for a later use could be colorcode for powerussage?  
#     autocolorscale=False,
#     showscale=False,
#     hoverinfo = 'none',
#     # text = df['ct']

# ))

# #Plot grid lines and set width, AC has to be treated specially 
# if carrier == 'AC':

#     for i in range(len(normal.lines.s_nom_opt)):
#         fig.add_trace(
#             go.Scattergeo(
#                 locationmode = 'geojson-id',# honestly no idea what this does
#                 lon = [df.loc[normal.lines.bus0[i]]['x'].round(3), df.loc[normal.lines.bus1[i]]['x'].round(3)],
#                 lat = [df.loc[normal.lines.bus0[i]]['y'].round(3), df.loc[normal.lines.bus1[i]]['y'].round(3)],
#                 mode = 'lines',
#                 line = dict(width = (normal.lines.s_nom_opt[i])/divisor,color = 'green'), #s_nom_opt nur Termische Kap = s_nom_opt * s_max_pu für net
#                 # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),  => Could be used later
#                 name="{} GW AC-Grid".format(int(round(normal.lines.s_nom_opt[i]/divisor)))
#             )
#         )
#     for i in range(len(dc.links.p_nom_opt)):
#         fig.add_trace(
#             go.Scattergeo(
#                 locationmode = 'geojson-id',# honestly no idea what this does
#                 lon = [df.loc[normal.links.bus0[i]]['x'].round(3), df.loc[normal.links.bus1[i]]['x'].round(3)],
#                 lat = [df.loc[normal.links.bus0[i]]['y'].round(3), df.loc[normal.links.bus1[i]]['y'].round(3)],
#                 mode = 'lines',
#                 line = dict(width = (dc.links.p_nom_opt[i])/divisor,color = 'red'),
#                 hoverinfo = 'text',
#                 text = dc.links.p_nom[i].round(0),
#                 # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
#                 name="{} GW DC-Grid".format(int(round(dc.links.p_nom_opt[i]/divisor)))
#             )
#         )

# else:

#     for i in range(len(normal.links.p_nom_opt)):
#         fig.add_trace(
#             go.Scattergeo(
#                 locationmode = 'geojson-id',# honestly no idea what this does
#                 lon = [df.loc[normal.links.bus0[i]]['x'].round(3), df.loc[normal.links.bus1[i]]['x'].round(3)],
#                 lat = [df.loc[normal.links.bus0[i]]['y'].round(3), df.loc[normal.links.bus1[i]]['y'].round(3)],
#                 mode = 'lines',
#                 line = dict(width = (normal.links.p_nom_opt[i])/divisor,color = 'red'),
#                 # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
#                 text = normal.links.p_nom[i],
#                 name="{} GW Pipeline".format(int(round(normal.links.p_nom_opt[i]/divisor)))

#             )
#         )
#     for i in range(len(retro.links.p_nom_opt)):
#         fig.add_trace(
#             go.Scattergeo(
#                 locationmode = 'geojson-id',# honestly no idea what this does
#                 lon = [df.loc[retro.links.bus0[i]]['x'].round(3), df.loc[retro.links.bus1[i]]['x'].round(3)],
#                 lat = [df.loc[retro.links.bus0[i]]['y'].round(3), df.loc[retro.links.bus1[i]]['y'].round(3)],
#                 mode = 'lines',
#                 line = dict(width = (retro.links.p_nom_opt[i])/divisor,color = 'blue'),
#                 hoverinfo = 'none',
#                 # text = retro.links.p_nom[i].round(0),
#                 name="{} GW retrofitted Pipeline".format(int(round(retro.links.p_nom_opt[i]/divisor)))
#                 # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
#             )
#         )

# #centralize on europe
# fig.update_geos(
#     visible=False, resolution=50, 
#     scope="europe",
#     # center=dict(lon=11, lat=40), # this part is to zone to AT
#     # projection_rotation=dict(lon=30, lat=30, roll=30),
#     # lataxis_range=[10,11], lonaxis_range=[33,41],
#     showcountries=True, countrycolor="Black",
#     #showsubunits=True, subunitcolor="Blue"
# )


# #Scale Figure 
# fig.update_layout(
#     title=f" Grid Capacity in {year}",
#     autosize=False,
#     width=1000,
#     height=1000,
#     #showlegend = True,
#     #hovermode='x unified',
#     #legend=dict(itemsizing='constant')
# )

# #the following part is only for legend edititng

# #create empty list to store legend labels
# labels_to_show_in_legend = []

# #as far as i understand plotly needs a geoobject to exist in the plot to be filtered in legend 
# #therefor find the closest existing Capacity to 3 and 10 GW and store them in List
# for s in (10,3):
#     if carrier == 'AC':
#         a = int(round(min(dc.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
#         labels_to_show_in_legend.append("{} GW DC-Grid".format(a))
#         b = int(round(min(normal.lines.s_nom_opt/divisor, key=lambda x:abs(x-s))))
#         labels_to_show_in_legend.append("{} GW AC-Grid".format(b))
#     elif carrier == 'gas':
#         c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
#         labels_to_show_in_legend.append("{} GW Pipeline".format(c))
#     else:
#         c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
#         labels_to_show_in_legend.append("{} GW Pipeline".format(c))
#         d = int(round(min(retro.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
#         labels_to_show_in_legend.append("{} GW retrofitted Pipeline".format(d))

# #create empty set
# labels = set()

# #check if trace name is not in set and in list, if true enable showlegend and store in set
# for trace in fig['data']: 
#     if ( not trace['name'] in labels) and (trace['name'] in labels_to_show_in_legend):
#         labels.add(trace['name'])
#         trace['showlegend'] = True
#         #print(trace['name'])   #enable print for troubleshooting
#     else:
#         trace['showlegend'] = False
            

# fig.show()


In [ ]:
#labels_to_show_in_legend, labels

In [ ]:
# for i in range(len(retro.links.p_nom_opt)):
#     print("{} GW retrofitted Pipeline".format(int(round(retro.links.p_nom_opt[i]/divisor))) , int(round(retro.links.p_nom_opt[i]/divisor)))

In [ ]:
def plot_grid(normal, retro, dc, carrier, year):
    """Plots a Geo. Figure in Plotly containing all clusters and a given grid
    Parameters
    ----------
    normal : Dataframe containing Links and Buses for AC and H2/gas grid, should be provided by get_grid_df funktion
    retro : Dataframe containing Links and Buses for retrofitted H2 grid, should be provided by get_grid_df funktion
    carrier : string (== "gas pipeline", "AC", "H2 pipeline")
        carrier (e.g. "gas pipeline", "AC") used to differenciate what grid to plot
    
    Returns
    -------
    Grid plot of a given carrier
    """
    carrier= carrier[:3].strip()          #this part is unnecessary here should be deleted as soon as get_grid_df is modified 
    df = normal.buses[normal.buses.carrier == carrier].copy()
    df['ct'] = df.index.str[:5]
    divisor = 1e3

    fig = go.Figure(data=go.Choropleth(
        #geojson=j_file,
        #featureidkey= 'properties.COUNTRY_ID',
        #locationmode='geojson-id', # honestly no idea what this does
        locations=df['ct'],
        z=df['v_nom'], # Placeholder Variable for a later use could be colorcode for powerussage?  
        autocolorscale=False,
        showscale=False,
        hoverinfo = 'none',
        # text = df['ct']

    ))

    #Plot grid lines and set width, AC has to be treated specially 
    if carrier == 'AC':

        for i in range(len(normal.lines.s_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    #locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[normal.lines.bus0[i]]['x'].round(3), df.loc[normal.lines.bus1[i]]['x'].round(3)],
                    lat = [df.loc[normal.lines.bus0[i]]['y'].round(3), df.loc[normal.lines.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (normal.lines.s_nom_opt[i])/divisor,color = 'green'), #s_nom_opt nur Termische Kap = s_nom_opt * s_max_pu für net
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),  => Could be used later
                    name="{} GW AC-Grid".format(int(round(normal.lines.s_nom_opt[i]/divisor)))
                )
            )
        for i in range(len(dc.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    #locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[normal.links.bus0[i]]['x'].round(3), df.loc[normal.links.bus1[i]]['x'].round(3)],
                    lat = [df.loc[normal.links.bus0[i]]['y'].round(3), df.loc[normal.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (dc.links.p_nom_opt[i])/divisor,color = 'red'),
                    hoverinfo = 'text',
                    text = dc.links.p_nom[i].round(0),
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
                    name="{} GW DC-Grid".format(int(round(dc.links.p_nom_opt[i]/divisor)))
                )
            )

    else:

        for i in range(len(normal.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    #locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[normal.links.bus0[i]]['x'].round(3), df.loc[normal.links.bus1[i]]['x'].round(3)],
                    lat = [df.loc[normal.links.bus0[i]]['y'].round(3), df.loc[normal.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (normal.links.p_nom_opt[i])/divisor,color = 'red'),
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
                    text = normal.links.p_nom[i],
                    name="{} GW Pipeline".format(int(round(normal.links.p_nom_opt[i]/divisor)))

                )
            )
        for i in range(len(retro.links.p_nom_opt)):
            fig.add_trace(
                go.Scattergeo(
                    #locationmode = 'geojson-id',# honestly no idea what this does
                    lon = [df.loc[retro.links.bus0[i]]['x'].round(3), df.loc[retro.links.bus1[i]]['x'].round(3)],
                    lat = [df.loc[retro.links.bus0[i]]['y'].round(3), df.loc[retro.links.bus1[i]]['y'].round(3)],
                    mode = 'lines',
                    line = dict(width = (retro.links.p_nom_opt[i])/divisor,color = 'blue'),
                    hoverinfo = 'none',
                    # text = retro.links.p_nom[i].round(0),
                    name="{} GW retrofitted Pipeline".format(int(round(retro.links.p_nom_opt[i]/divisor)))
                    # opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
                )
            )

    #centralize on europe
    fig.update_geos(
        visible=False, resolution=50, 
        scope="europe",
        # center=dict(lon=11, lat=40), # this part is to zone to AT
        # projection_rotation=dict(lon=30, lat=30, roll=30),
        # lataxis_range=[10,11], lonaxis_range=[33,41],
        showcountries=True, countrycolor="Black",
        #showsubunits=True, subunitcolor="Blue"
    )


    #Scale Figure 
    fig.update_layout(
        title=f" Grid Capacity in {year}",
        autosize=False,
        width=1000,
        height=1000,
        #showlegend = True,
        #hovermode='x unified',
        #legend=dict(itemsizing='constant')
    )

    #the following part is only for legend edititng

    #create empty list to store legend labels
    labels_to_show_in_legend = []

    #as far as i understand plotly needs a geoobject to exist in the plot to be filtered in legend 
    #therefor find the closest existing Capacity to 3 and 10 GW and store them in List
    for s in (10,3):
        if carrier == 'AC':
            a = int(round(min(dc.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW DC-Grid".format(a))
            b = int(round(min(normal.lines.s_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW AC-Grid".format(b))
        elif carrier == 'gas':
            c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW Pipeline".format(c))
        else:
            c = int(round(min(normal.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW Pipeline".format(c))
            d = int(round(min(retro.links.p_nom_opt/divisor, key=lambda x:abs(x-s))))
            labels_to_show_in_legend.append("{} GW retrofitted Pipeline".format(d))

    #create empty set
    labels = set()

    #check if trace name is not in set and in list, if true enable showlegend and store in set
    for trace in fig['data']: 
        if ( not trace['name'] in labels) and (trace['name'] in labels_to_show_in_legend):
            labels.add(trace['name'])
            trace['showlegend'] = True
            #print(trace['name'])   #enable print for troubleshooting
        else:
            trace['showlegend'] = False
            
    return(fig)

In [ ]:
#pip install dash-bootstrap-components

In [ ]:
from dash import Dash, dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px

# incorporate data into app
#df = px.data.medals_long()

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
mytitle = dcc.Markdown(children='# TEST TEST')
mygraph = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=['H2 pipeline', 'gas pipeline', "AC"],
                        value='H2 pipeline',  # initial value displayed when page first loads
                        clearable=False)
dropdown_year = dcc.Dropdown(options=['2015','2020','2030','2040','2050'],
                        value='2015',  # initial value displayed when page first loads
                        clearable=False)

# Customize your own Layout
app.layout = dbc.Container([mytitle, dropdown, dropdown_year, mygraph])

# Callback allows components to interact
@app.callback(
    Output(mygraph, component_property='figure'),
    Input(dropdown, component_property='value'),
    Input(dropdown_year, component_property='value')
)
def update_graph(user_input, dropdown_year):  # function arguments come from the component property of the Input
    carrier = user_input
    n = networks[dropdown_year]
    normal,retro,dc = get_grid_df(n,carrier)
    fig = plot_grid(normal, retro, dc, carrier, dropdown_year)
    return fig  # returned objects are assigned to the component property of the Output




In [ ]:
# Run app
if __name__=='__main__':
    app.run_server( port=9998)

## Sachen die noch fehlen:

1) Österreich richtig plotten
2) Legende
3) AC und DC addieren


In [ ]:
handles = []
labels = []
line_factor = 1e3
for s in (10, 3): # these values define the legend, change for different scaling
    handles.append(plt.Line2D([0], [0], color="green",
                                linewidth=s * 1e3 / line_factor))
    labels.append("{} GW".format(s))

In [ ]:
handles